In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import datetime
import os

In [0]:
datasets,info=tfds.load('amazon_us_reviews/Major_Appliances_v1_00', with_info=True, split=('train[:70%]', 'train[70%:85%]','train[85%:]'))

In [3]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [0]:
train_dataset=datasets[0]
test_dataset=datasets[1]
val_dataset=datasets[2]

In [5]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'48220985'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Major Appliances'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B002ZD5I16'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'476878110'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Whirlpool Part Number 4388736: DRUM'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'Just was described, works great, fast shipping!!'>, 'review_date': <tf.Tensor: shape=(), dtype=string, numpy=b'2015-04-08'>, 'review_headline': <tf.Tensor: shape=(), dtype=string, numpy=b'Perfect Fit'>, 'review_id': <tf.Tensor: shape=(), dtype=string, numpy=b'R2PIXJFK315V9F'>, 'star_rating': <tf.Tensor: shape=(), dtype=int32, numpy=5>, 'total_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'ver

In [0]:
@tf.function
def target(x):
  if x>3:
    return 1
  else:
    return 0  

In [7]:
for reviews in train_dataset.take(5):
  review_text=reviews['data']
  print(review_text.get('review_body').numpy())
  star_rating=review_text.get('star_rating')
  print (star_rating)
  print (target(star_rating).numpy())

b'Just was described, works great, fast shipping!!'
tf.Tensor(5, shape=(), dtype=int32)
1
b'Amazing replacement! I kept buying the exspensive name brand filters and decided to try this out. You could not tell the difference! Awesome product!'
tf.Tensor(5, shape=(), dtype=int32)
1
b"The round rubbers that holds the spin dryer usually break down into pieces that your spin dryer doesn't turned due to balance.. Good product keeping your clothes dry quickly, but bad because of the rubber shies break up.."
tf.Tensor(2, shape=(), dtype=int32)
0
b"Absolutely love my new laundry pod! The only thing that would make me love it more was if there was a left-handed option available. The green grip on the cover is awesome for my right-handed boyfriend; not so much for me. Granted, I could use it backwards, but it's so much cuter front facing!"
tf.Tensor(5, shape=(), dtype=int32)
1
b'We have had this refrigerator for 1 year.  The repairmen have come out 3 times and the ice maker/water dispenser still 

In [0]:
def fetch(tensor):
  text=tensor['data'].get('review_body')
  label=target(tensor['data'].get('star_rating'))
  return text,label

In [0]:
train_data_f=train_dataset.map(fetch)
test_data_f=test_dataset.map(fetch)
val_data_f=val_dataset.map(fetch)

In [10]:
train_data, train_labels=next(iter(train_data_f.batch(5)))
train_data

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'Just was described, works great, fast shipping!!',
       b'Amazing replacement! I kept buying the exspensive name brand filters and decided to try this out. You could not tell the difference! Awesome product!',
       b"The round rubbers that holds the spin dryer usually break down into pieces that your spin dryer doesn't turned due to balance.. Good product keeping your clothes dry quickly, but bad because of the rubber shies break up..",
       b"Absolutely love my new laundry pod! The only thing that would make me love it more was if there was a left-handed option available. The green grip on the cover is awesome for my right-handed boyfriend; not so much for me. Granted, I could use it backwards, but it's so much cuter front facing!",
       b'We have had this refrigerator for 1 year.  The repairmen have come out 3 times and the ice maker/water dispenser still is not right.  Water freezes up in the top of the refrigerator; ice 

In [11]:
train_labels

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 1, 0, 1, 0], dtype=int32)>

In [19]:
embedding="https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer=hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-9.51282680e-02, -9.18786898e-02, -5.37700318e-02,
         1.08619660e-01,  4.07037511e-02, -5.80801852e-02,
        -5.21476083e-02,  3.19720097e-02,  1.56951353e-01,
        -9.36029851e-02, -5.06434217e-02,  1.50605990e-03,
         6.62063584e-02, -9.65742245e-02, -6.13453947e-02,
         2.53834147e-02,  3.55761275e-02, -9.14261211e-03,
        -5.73733822e-02, -5.07911257e-02,  6.43319190e-02,
        -1.17676198e-01,  6.34124852e-04,  1.81849897e-02,
         7.11547211e-03, -2.71910988e-02,  7.46759251e-02,
        -1.56106636e-01,  1.66973442e-01, -4.39601801e-02,
        -8.42082351e-02, -9.65642333e-02, -6.22393563e-02,
         1.67741507e-01,  3.37394774e-02,  1.20292995e-02,
         3.07388641e-02, -5.44720143e-02, -3.84759828e-02,
         7.27326497e-02,  1.50642246e-01,  1.24470912e-01,
         2.02967450e-01, -1.53443784e-01,  7.29663223e-02,
         8.31599161e-02, -4.33836803e-02, -9.69492644e-02,
      

In [22]:
model=tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 124,646,849
Trainable params: 124,646,849
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']
              )

In [0]:
train_data_f=train_data_f.shuffle(70000).batch(512)
val_data_f=val_data_f.batch(512)

In [25]:
history=model.fit(train_data_f,
                  epochs=4,
                  validation_data=val_data_f,
                  verbose=1)

Epoch 1/4
133/133 [==============================] - 177s 1s/step - loss: 0.5478 - accuracy: 0.8118 - val_loss: 0.5030 - val_accuracy: 0.8773
Epoch 2/4
133/133 [==============================] - 176s 1s/step - loss: 0.4917 - accuracy: 0.8955 - val_loss: 0.4959 - val_accuracy: 0.8815
Epoch 3/4
133/133 [==============================] - 175s 1s/step - loss: 0.4808 - accuracy: 0.9169 - val_loss: 0.4951 - val_accuracy: 0.8803
Epoch 4/4
133/133 [==============================] - 177s 1s/step - loss: 0.4743 - accuracy: 0.9294 - val_loss: 0.4952 - val_accuracy: 0.8810


In [27]:
results=model.evaluate(test_data_f.batch(512),verbose=2)
print(results)

29/29 - 6s - loss: 0.4954 - accuracy: 0.8821
[0.4954421818256378, 0.8821465373039246]


In [28]:
for f0,f1 in test_data_f.batch(15000):
  y=f1
  y_pred=tf.where(model.predict(f0)>0.5,1,0)
  print(tf.math.confusion_matrix(labels=y, predictions=y_pred, num_classes=2))

tf.Tensor(
[[4140  698]
 [1015 8682]], shape=(2, 2), dtype=int32)
